In [1]:
import mitsuba as mi 
mi.set_variant("cuda_ad_rgb")
from PDE2D.Coefficient import *
from PDE2D.utils import *
from matplotlib import pyplot as plt, patches
bbox = [[-2, -2],[2, 2]]
from mitsuba import Point2f, Float, Texture2f, Texture1f

In [2]:
image = np.array([[1, 3, 3, 3, 2, 4, 5, 2],
                  [4, 4, 3, 8, 8, 1, 1, 1],
                  [8, 4, 9, 6, 0, 1, 1, 5],
                  [1, 0, 4, 1, 1, 1, 1, 3],
                  [2, 0, 3, 1, 1, 1, 1, 5],
                  [3, 0, 0, 3, 1, 1, 2, 1],
                  [3, 8, 8, 8, 8, 8, 4, 4],
                  [4, 2, 3, 5, 1, 1, 5, 1]])
coeff = TextureCoefficient("test", bbox, image)

In [3]:

resolution = [256, 256]
spp = 1024
fd_step = 1e-3
points = create_image_points(bbox, resolution, spp)

In [ ]:

fnc_vals = coeff.get_value(points)
image_, tensor_ = create_image_from_result(fnc_vals,  resolution)
fig, (ax1) = plt.subplots(1,1, figsize=[5, 5])
plot_image(image_, ax1)
ax1.set_title("FNC")

In [5]:

fnc_vals_fdx = coeff.get_value(points +Point2f(1,0) * fd_step)
fnc_vals_fdy = coeff.get_value(points +Point2f(0,1) * fd_step)
fnc_vals_fdx_ = coeff.get_value(points -Point2f(1,0) * fd_step)
fnc_vals_fdy_ = coeff.get_value(points -Point2f(0,1) * fd_step)
image, tensor = create_image_from_result(fnc_vals, resolution)
image_fdx, tensor_fdx = create_image_from_result((fnc_vals_fdx - fnc_vals_fdx_) / (2 * fd_step), resolution)
image_fdy, tensor_fdy = create_image_from_result((fnc_vals_fdy - fnc_vals_fdy_) / (2 * fd_step),  resolution)
grad, laplacian = coeff.get_grad_laplacian(points)
grad_x, grad_x_tensor = create_image_from_result(grad[0], resolution)
grad_y, grad_x_tensor  = create_image_from_result(grad[1],resolution)


laplacian_fd = ((fnc_vals_fdx + fnc_vals_fdx_ - 2 * fnc_vals) / dr.square(fd_step) +
                (fnc_vals_fdy + fnc_vals_fdy_ - 2 * fnc_vals) / dr.square(fd_step))
laplacian_fd_im, laplacian_fd_tensor = create_image_from_result(laplacian_fd, resolution)
laplacian_fnc_im, laplacian_fnc_tensor = create_image_from_result(laplacian, resolution)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=[11, 5])
plot_image(grad_x, ax1)
ax1.set_title("FNC")
plot_image(image_fdx, ax2)
ax2.set_title("FD")
fig.suptitle("Grad X")

In [ ]:
sketch_diff(grad_x, image_fdx)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=[11, 5])
plot_image(grad_y, ax1)
ax1.set_title("FNC")
plot_image(image_fdy, ax2)
ax2.set_title("FD")
fig.suptitle("Grad Y")

In [ ]:
sketch_diff(grad_y, image_fdy)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=[11, 5])
plot_image(laplacian_fnc_im, ax1)
ax1.set_title("FNC")
plot_image(laplacian_fd_im, ax2)
ax2.set_title("FD")
fig.suptitle("Laplacian")

In [ ]:
sketch_diff(laplacian_fd_im, laplacian_fnc_im)